In [9]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing  import StandardScaler

class SOCPredictor:
    def __init__(self, model_path, scaler_path):
        # 硬件配置检测
        self.device  = torch.device("cuda"  if torch.cuda.is_available()  else "cpu")

        # 模型结构定义（必须与训练时完全一致）
        self.model  = self._define_model_structure().to(self.device)
        self.model.load_state_dict(torch.load(model_path,  map_location=self.device))
        self.model.eval()

        # 加载标准化器
        self.scaler  = self._load_scaler(scaler_path)

    def _define_model_structure(self):
        """模型结构定义（需与训练代码完全一致）"""
        return nn.Sequential(
            nn.Linear(3, 128),  # 输入层 -> 隐藏层1
            nn.LeakyReLU(0.01),  # 负区间保留0.01斜率
            nn.BatchNorm1d(128),  # 批量归一化加速收敛
            nn.Linear(128, 64),  # 隐藏层1 -> 隐藏层2
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),  # 随机丢弃10%神经元
            nn.Linear(64, 32),  # 隐藏层2 -> 隐藏层3
            nn.LeakyReLU(0.01),
            nn.Linear(32, 1)  # 输出层
        )

    def _load_scaler(self, path):
        """加载标准化器对象"""
        scaler = StandardScaler()
        scaler.mean_,  scaler.scale_  = np.load(path,  allow_pickle=True)
        return scaler

    def predict(self, input_data):
        """
        输入数据格式要求：
        - DataFrame或可转换为DataFrame的数据结构
        - 列顺序：Voltage, Current, Temperature
        """
        # 数据校验
        assert input_data.shape[1]  == 3, "输入数据需要包含3个特征"

        # 数据预处理
        processed_data = self.scaler.transform(input_data)
        tensor_data = torch.tensor(processed_data,  dtype=torch.float32).to(self.device)

        # 执行预测
        with torch.no_grad():
            predictions = self.model(tensor_data).cpu().numpy().flatten()

        return predictions

In [10]:
# 示例输入数据（需替换为实际数据）
new_data = pd.DataFrame({
    'Voltage': [3.7, 3.5, 3.8],
    'Current': [2.1, 1.8, 2.3],
    'Temperature': [25.0, 28.5, 23.7]
})

# 数据顺序验证
input_data = new_data[['Voltage', 'Current', 'Temperature']].values

In [11]:
predictor = SOCPredictor(
    model_path="best_model.pth",
    scaler_path="scaler_params.npy"   # 需保存训练时的scaler参数
)

C:\Users\admin\AppData\Local\Temp\ipykernel_28112\3584257498.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path,  map_loca

RuntimeError: Error(s) in loading state_dict for Sequential:
	Missing key(s) in state_dict: "0.weight", "0.bias", "2.weight", "2.bias", "2.running_mean", "2.running_var", "3.weight", "3.bias", "6.weight", "6.bias", "8.weight", "8.bias". 
	Unexpected key(s) in state_dict: "net.0.weight", "net.0.bias", "net.2.weight", "net.2.bias", "net.2.running_mean", "net.2.running_var", "net.2.num_batches_tracked", "net.3.weight", "net.3.bias", "net.6.weight", "net.6.bias", "net.8.weight", "net.8.bias". 